In [1]:
import cv2  # 이미지 데이터 ndarray
import os
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torch.utils.data import random_split
from torchvision.datasets import ImageFolder
from torchvision.transforms import v2
from torchvision import transforms
from torch.utils.data import DataLoader,Dataset
import torchvision.models as models
from torchinfo import summary


import pickle       # 피클(pickle)된 데이터를 역직렬화(deserialize)**하는 함수


from PIL import Image   # 이미지 데이터 JPGImage

In [9]:
def load_image(image_path):
    image = Image.open(image_path).convert('RGB')
    return image

preprocess = transforms.Compose([
    # transforms.Resize((224, 224)),  # 이미지 크기 조정 (모델에 따라 조정 필요)
    transforms.ToTensor()          # 텐서로 변환
    # transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # 정규화 (VGG16 모델에 맞춘 값)
])

def predict(image_path, model):
    image = load_image(image_path)   # 이미지 로드
    image = preprocess(image)         # 전처리 적용
    image = image.unsqueeze(0)        # 배치 차원 추가
    image = image         # 디바이스로 이동

    with torch.no_grad():             # 그라디언트 계산 비활성화
        outputs = model(image)        # 모델에 이미지 입력
        probabilities = torch.softmax(outputs, dim=1)  # 소프트맥스 적용
        # print(probabilities)          # 각 클래스에 대한 확률 출력
        _, predicted = torch.max(probabilities.data, 1)  # 예측 클래스

    return probabilities, predicted.item()

# 모델 로드

model = torch.load(r'C:\Git\KDT\BigData\Project\OpenCv\models\6class_final_model_train_wbs.pth',weights_only=False ,map_location=torch.device('cpu'))  # 가중치 불러오기
model.eval()  # 모델 평가 모드로 전환
# model
# 예측할 이미지 경로
image_path = r'C:\Git\KDT\BigData\Project\OpenCv\Data_all\test\BearGGu (7).jpg'
# print(r"C:\Users\Doyeon\Desktop\KDT\OpenCV\test\u76.jpg")
# 예측 수행
predicted_class = predict(image_path, model)
print(f'Predicted class: {predicted_class}')

Predicted class: (tensor([[1.0000e+00, 9.5460e-09, 3.6003e-09, 2.6212e-07, 1.0860e-09, 8.8221e-08]]), 0)


In [ ]:
model = models.resnet18(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 6)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if device == 'cuda':
    print("cuda")
else:
    print("cpu")
model = model.to(device)

def initialize_weights(m):
    if isinstance(m, nn.Linear):
        nn.init.kaiming_normal_(m.weight)
        nn.init.constant_(m.bias, 0)

model.apply(initialize_weights)
# 손실 함수와 옵티마이저 설정
class_weights = torch.tensor([1.0, 1.0, 1.0, 1.0, 1.0, 1.0]).to(device)
criterion = nn.CrossEntropyLoss(weight=class_weights)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)